In [ ]:
import pandas as pd
import folium
import plotly.express as px

In [ ]:
# %% Đọc dữ liệu mẫu (hoặc đọc từ MinIO)
df = pd.read_parquet("s3://nyc-taxi-driver/processed/taxi_demand_features") # Cần đường dẫn thực tế

In [ ]:
# %% 1. Biểu đồ đường: Nhu cầu theo giờ
hourly_demand = df.groupby("pickup_hour")["trip_count"].sum().reset_index()
fig = px.line(hourly_demand, x="pickup_hour", y="trip_count", title="Tổng nhu cầu Taxi theo giờ trong ngày")
fig.show()

In [ ]:
# %% 2. Bản đồ nhiệt (Heatmap) với Folium
# Cần file shapefile hoặc GeoJSON của NYC Taxi Zones để map LocationID với tọa độ
# (Bạn có thể tải NYC Taxi Zones GeoJSON trên mạng)
import requests
geojson_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.json"
nyc_geo = requests.get(geojson_url).json()

In [ ]:
# Tổng hợp theo location
loc_demand = df.groupby("PULocationID")["trip_count"].sum().reset_index()

m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

folium.Choropleth(
    geo_data=nyc_geo,
    name="choropleth",
    data=loc_demand,
    columns=["PULocationID", "trip_count"],
    key_on="feature.properties.LocationID",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Taxi Demand"
).add_to(m)

m